In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
def get_transfer_spends(url):
    tx_array = []
    headers = {"User-Agent":"Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    tables = soup.findAll("table")
    for i in range(0,6):
        footer = tables[i].find('tfoot')
        footer.find('td').contents[0]
        x=footer.find('td').contents[0]
        tx_array.append(x)
    return tx_array

In [3]:
headers = {"User-Agent":"Mozilla/5.0"}
response = requests.get('https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1', headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
tables = soup.findAll("table")

In [5]:
team_data = tables[3].findAll("tbody")
team_array=[]
for i in team_data[0].findAll("td", {'class':'zentriert no-border-rechts'}):
    team_name = (i.find('a').find('img')['alt'])
    team_array.append(team_name)
    team_link = (i.find('a')['href'])
    team_link.find('/saison')
    team_short_link = team_link[0:team_link.find('/saison')]
    transfer_link = team_short_link.replace('startseite','alletransfers',1)
    full_tx_link = 'https://www.transfermarkt.com' + transfer_link
    spends = get_transfer_spends(full_tx_link)
    for j in spends:
        team_array.append(j)

In [6]:
team_df = pd.DataFrame()
for i in range(0,len(team_array)):
    team_series = pd.Series(team_array[i:i+7])
    team_df = team_df.append(team_series,ignore_index=True)
    i = i+ 7
transfer_df = team_df.iloc[[0,7,14,21,28,35,42,49,56,63,70,77,84,91,98,105,112,119,126,133]]
transfer_df = transfer_df.reset_index()
transfer_df = transfer_df.drop(columns=['index'])
transfer_df['League'] = 'La Liga'

In [17]:
def generate_xfer_df(league,league_url):
    headers = {"User-Agent":"Mozilla/5.0"}
    response = requests.get(league_url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    tables = soup.findAll("table")
    team_data = tables[3].findAll("tbody")
    team_array=[]
    for i in team_data[0].findAll("td", {'class':'zentriert no-border-rechts'}):
        team_name = (i.find('a').find('img')['alt'])
        team_array.append(team_name)
        team_link = (i.find('a')['href'])
        team_link.find('/saison')
        team_short_link = team_link[0:team_link.find('/saison')]
        transfer_link = team_short_link.replace('startseite','alletransfers',1)
        full_tx_link = 'https://www.transfermarkt.com' + transfer_link
        spends = get_transfer_spends(full_tx_link)
        for j in spends:
            team_array.append(j)
    team_df = pd.DataFrame()
    for i in range(0,len(team_array)):
        team_series = pd.Series(team_array[i:i+7])
        team_df = team_df.append(team_series,ignore_index=True)
        i = i+ 7
    transfer_df = team_df.iloc[[0,7,14,21,28,35,42,49,56,63,70,77,84,91,98,105,112,119,126,133]]
    transfer_df = transfer_df.reset_index()
    transfer_df = transfer_df.drop(columns=['index'])
    transfer_df['League'] = league
    return transfer_df

In [18]:
x=generate_xfer_df('La Liga','https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1')

In [19]:
x

,0,1,2,3,4,5,6,League
0,Real Madrid,€307.50m,€130.00m,€162.75m,€134.10m,€40.50m,€128.50m,La Liga
1,FC Barcelona,€255.00m,€156.40m,€129.10m,€134.05m,€374.50m,€232.50m,La Liga
2,Atlético Madrid,€56.00m,€243.50m,€313.10m,€160.50m,€64.90m,€102.00m,La Liga
3,Valencia CF,€12.00m,€75.00m,€45.00m,€127.00m,€68.80m,€55.00m,La Liga
4,Real Sociedad,€250k,€21.25m,€9.50m,€12.00m,€30.80m,€21.50m,La Liga
5,Real Betis Balompié,€95.75m,€69.50m,€42.50m,€48.25m,€38.50m,€27.07m,La Liga
6,Sevilla FC,0,€158.75m,€101.20m,€79.25m,€91.25m,€78.05m,La Liga
7,Celta de Vigo,€22.90m,€19.65m,€29.50m,€34.80m,€25.80m,€13.20m,La Liga
8,Athletic Bilbao,0,0,0,€32.90m,€80.00m,€32.00m,La Liga
9,Villarreal CF,€21.80m,€62.00m,€92.90m,€59.80m,€46.10m,€67.30m,La Liga


In [10]:
transfer_df

,0,1,2,3,4,5,6,League
0,Real Madrid,€307.50m,€130.00m,€162.75m,€134.10m,€40.50m,€128.50m,La Liga
1,FC Barcelona,€255.00m,€156.40m,€129.10m,€134.05m,€374.50m,€232.50m,La Liga
2,Atlético Madrid,€56.00m,€243.50m,€313.10m,€160.50m,€64.90m,€102.00m,La Liga
3,Valencia CF,€12.00m,€75.00m,€45.00m,€127.00m,€68.80m,€55.00m,La Liga
4,Real Sociedad,€250k,€21.25m,€9.50m,€12.00m,€30.80m,€21.50m,La Liga
5,Real Betis Balompié,€95.75m,€69.50m,€42.50m,€48.25m,€38.50m,€27.07m,La Liga
6,Sevilla FC,0,€158.75m,€101.20m,€79.25m,€91.25m,€78.05m,La Liga
7,Celta de Vigo,€22.90m,€19.65m,€29.50m,€34.80m,€25.80m,€13.20m,La Liga
8,Athletic Bilbao,0,0,0,€32.90m,€80.00m,€32.00m,La Liga
9,Villarreal CF,€21.80m,€62.00m,€92.90m,€59.80m,€46.10m,€67.30m,La Liga


In [20]:
epl_df = generate_xfer_df('Premier League','https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1')

In [21]:
epl_df

,0,1,2,3,4,5,6,League
0,Manchester City,0,€168.00m,€69.00m,€78.59m,€54.10m,€317.50m,Premier League
1,Liverpool FC,€1.90m,€41.60m,€182.20m,€41.10m,€173.88m,€194.50m,Premier League
2,Tottenham Hotspur,€114.00m,€35.00m,0,€5.35m,€121.50m,€103.80m,Premier League
3,Chelsea FC,€56.00m,€45.00m,€145.75m,€208.80m,€69.55m,€260.50m,Premier League
4,Manchester United,€159.00m,€69.50m,€82.70m,€30.55m,€198.40m,€45.50m,Premier League
5,Arsenal FC,€152.40m,€53.55m,€80.15m,€7.90m,€152.85m,€158.00m,Premier League
6,Everton FC,€119.90m,€80.50m,€99.80m,€28.65m,€203.20m,€126.38m,Premier League
7,Leicester City,€106.10m,€87.30m,€114.60m,€95.80m,€87.70m,€47.90m,Premier League
8,West Ham United,0,€79.70m,€53.38m,€93.90m,€14.87m,€56.80m,Premier League
9,AFC Bournemouth,€56.45m,€34.50m,€89.10m,€20.20m,€34.30m,0,Premier League


In [13]:
transfer_df

,0,1,2,3,4,5,6,League
0,Real Madrid,€307.50m,€130.00m,€162.75m,€134.10m,€40.50m,€128.50m,La Liga
1,FC Barcelona,€255.00m,€156.40m,€129.10m,€134.05m,€374.50m,€232.50m,La Liga
2,Atlético Madrid,€56.00m,€243.50m,€313.10m,€160.50m,€64.90m,€102.00m,La Liga
3,Valencia CF,€12.00m,€75.00m,€45.00m,€127.00m,€68.80m,€55.00m,La Liga
4,Real Sociedad,€250k,€21.25m,€9.50m,€12.00m,€30.80m,€21.50m,La Liga
5,Real Betis Balompié,€95.75m,€69.50m,€42.50m,€48.25m,€38.50m,€27.07m,La Liga
6,Sevilla FC,0,€158.75m,€101.20m,€79.25m,€91.25m,€78.05m,La Liga
7,Celta de Vigo,€22.90m,€19.65m,€29.50m,€34.80m,€25.80m,€13.20m,La Liga
8,Athletic Bilbao,0,0,0,€32.90m,€80.00m,€32.00m,La Liga
9,Villarreal CF,€21.80m,€62.00m,€92.90m,€59.80m,€46.10m,€67.30m,La Liga


In [14]:
headers = {"User-Agent":"Mozilla/5.0"}
response = requests.get('https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1', headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
tables = soup.findAll("table")
team_data = tables[3].findAll("tbody")
team_array=[]
for i in team_data[0].findAll("td", {'class':'zentriert no-border-rechts'}):
    team_name = (i.find('a').find('img')['alt'])
    team_array.append(team_name)
    team_link = (i.find('a')['href'])
    team_link.find('/saison')
    team_short_link = team_link[0:team_link.find('/saison')]
    transfer_link = team_short_link.replace('startseite','alletransfers',1)
    full_tx_link = 'https://www.transfermarkt.com' + transfer_link
    spends = get_transfer_spends(full_tx_link)
    for j in spends:
        team_array.append(j)
team_df = pd.DataFrame()
for i in range(0,len(team_array)):
    team_series = pd.Series(team_array[i:i+7])
    team_df = team_df.append(team_series,ignore_index=True)
    i = i+ 7
transfer_df = team_df.iloc[[0,7,14,21,28,35,42,49,56,63,70,77,84,91,98,105,112,119,126,133]]
transfer_df = transfer_df.reset_index()
transfer_df = transfer_df.drop(columns=['index'])
transfer_df['League'] = 'La Liga'

In [15]:
transfer_df

,0,1,2,3,4,5,6,League
0,Manchester City,0,€168.00m,€69.00m,€78.59m,€54.10m,€317.50m,La Liga
1,Liverpool FC,€1.90m,€41.60m,€182.20m,€41.10m,€173.88m,€194.50m,La Liga
2,Tottenham Hotspur,€114.00m,€35.00m,0,€5.35m,€121.50m,€103.80m,La Liga
3,Chelsea FC,€56.00m,€45.00m,€145.75m,€208.80m,€69.55m,€260.50m,La Liga
4,Manchester United,€159.00m,€69.50m,€82.70m,€30.55m,€198.40m,€45.50m,La Liga
5,Arsenal FC,€152.40m,€53.55m,€80.15m,€7.90m,€152.85m,€158.00m,La Liga
6,Everton FC,€119.90m,€80.50m,€99.80m,€28.65m,€203.20m,€126.38m,La Liga
7,Leicester City,€106.10m,€87.30m,€114.60m,€95.80m,€87.70m,€47.90m,La Liga
8,West Ham United,0,€79.70m,€53.38m,€93.90m,€14.87m,€56.80m,La Liga
9,AFC Bournemouth,€56.45m,€34.50m,€89.10m,€20.20m,€34.30m,0,La Liga
